<a href="https://colab.research.google.com/github/NinaOmani/S2S_Precipitation_Forecasting/blob/main/A_Hindcast_Reanalysis_Predictors_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hindcast and Reanalysis Predictor Preparation

**Author**: Nina Omani  
**Created**: 01-28-2025

---

## Notebook Description

This notebook prepares predictors for seasonal precipitation forecasting using:

- **ECMWF** Hindcast Q850_based Weather Types (WT)   
- **ERA5** Reanalysis Q850_based Weather Types (WT)
- **ECMWF** Hindcast Climate Variables    
- **ERA5** Reanalysis Climate Variables  
- **PRISM** Precipitation Observations  
- **ENSO** Indices (MEI, SOI, dSST3.4)  
- **Antecedent Precipitation** Calculations  

It merges and processes these datasets for AZ and NM regions across monsoon seasons, producing a final merged dataset of relevant predictors for precipitation forecasting.

---

## Workflow Overview

1. Load and preprocess WT and PRISM precipitation data  
2. Merge with ENSO indices  
3. Aggregate by season and lead month  
4. Process ERA and ECMWF climate variables  
5. Calculate antecedent precipitation  
6. Merge all into a unified predictor dataset

---


In [2]:
!pip install pandas numpy scikit-learn scipy matplotlib
!pip install gdown

In [3]:
import os
import gdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from scipy.spatial.distance import mahalanobis

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Define forecast and historic years
hindcast_years = list(range(1981, 2023))  # 1981 to 2023 (last update: Feb 2024)
historic_years = list(range(1985, 2022))  # 1985 to 2022

In [6]:
model_dir = "ECMWF"
os.makedirs(model_dir, exist_ok=True)

In [7]:
# Define a region
regions = ["AZ_West", "AZ_East", "NM_South", "NM_North"]

### **PRISM + ECMWF WT + ERA WT**

The file `paired_PRISM_ECMWF_WT_1981_2023.csv` contains combined Weather Type data (from ECMWF and ERA reanalysis) and regional PRISM precipitation data.

**Source Notebook**: `WT_File_gather.ipynb`


In [8]:
# Load the dataset
df_prism_wt = pd.read_csv("paired_PRISM_ECMWF_WT_1981_2023.csv")
df_prism_wt["Month"] = pd.to_numeric(df_prism_wt["Month"], errors='coerce')
df_prism_wt["Year"] = pd.to_numeric(df_prism_wt["Year"], errors='coerce').astype("Int64")
df_prism_wt.to_csv("df_prism_wt.csv", index=False)

Downloading...
From: https://drive.google.com/uc?id=1Gjwgcr5HWnBCxUfhHFfcKDlmwN_ti81I
To: /content/paired_PRISM_ECMWF_WT_1981_2023.csv
100%|██████████| 301k/301k [00:00<00:00, 65.5MB/s]


📈 **ENSO Data**  
`ENSO_data.csv` contains ENSO climate indices used in the analysis.  
📂 **Source Notebook**: `Climate_indices.ipynb` in R


In [9]:
# Load the dataset
enso = pd.read_csv("ENSO_data.csv")

month_map = {
    "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6,
    "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12
}

if 'Date' in enso.columns:
    enso.drop(columns=['Date'], inplace=True)

enso["Month"] = enso["Month"].map(month_map)
enso["Year"] = pd.to_numeric(enso["Year"], errors='coerce').astype("Int64")
enso["Month"] = pd.to_numeric(enso["Month"], errors='coerce').astype("Int64")

Downloading...
From: https://drive.google.com/uc?id=1yBUuG60_7_nrSUsLdxnRb2c2HyuHByUd
To: /content/ENSO_data.csv
100%|██████████| 44.5k/44.5k [00:00<00:00, 36.5MB/s]


**Combine ENSO and WT**

In [10]:
# Convert Year and Month columns to integer type in both datasets for consistency
df2= enso.copy()
df1= df_prism_wt.copy()

enso_WT = pd.merge(df1, df2, on=['Year', 'Month'], how='outer')
enso_WT = enso_WT.sort_values(by=['Year', 'lead_month', 'Month', 'region'], ascending=[True, True, True, True])
enso_WT = enso_WT[enso_WT['Year'] > 1980]

**Process ENSO Features**

In [11]:
enso_features = enso_WT.copy()
lead_months = [4, 5, 6, 7, 8]
new_cols = []
enso_predictors = ["SOI", "MEI", "dSST3.4"]

for pred in enso_predictors:
    # Define column names for observed and forecast values
    obs_col = f"ant_{pred}_obs"
    for_col = f"ant_{pred}_for"

    # Assign observed values
    enso_features[obs_col] = enso_features[pred]

    for lead_month in lead_months:
        antecedent_month = lead_month - 1

        for year in enso_features["Year"].unique():
            antecedent_value = enso_features.loc[
                (enso_features["Year"] == year) & (enso_features["Month"] == antecedent_month), pred
            ].values

            if len(antecedent_value) > 0:  # Ensure there's a valid antecedent value
                enso_features.loc[
                    (enso_features["Year"] == year) &
                    (enso_features["Month"].isin([6, 7, 8, 9, 10])) &
                    (enso_features["lead_month"] == lead_month),
                    for_col
                ] = antecedent_value[0]

    new_cols.extend([obs_col, for_col])

# Save the ENSO features
enso_features.to_csv("ENSO_Features.csv", index=False)
print(f"✅ ENSO Features successfully saved to: ENSO_Features.csv")


✅ ENSO Features successfully saved to: ENSO_Features.csv


**Seasonal ENSO features**

In [12]:
df= enso_features.copy()
df.columns = df.columns.str.lower()

# Define seasons
season_definitions = {
    "Jun": (df["lead_month"].isin([4, 5, 6])) & (df["month"] == 6),
    "Jul": (df["lead_month"].isin([4, 5, 6, 7])) & (df["month"] == 7),
    "Aug": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"] == 8),
    "Sep": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"] == 9),
    "Oct": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"] == 10)}

final_data = pd.DataFrame()

rename_dict = {
    "areal_mean_precipitation_mm_day": "avgPCP_mmDay"
}

final_data = pd.concat([
    df[condition].assign(season=season).rename(columns={k: v for k, v in rename_dict.items() if k in df.columns})
    for season, condition in season_definitions.items()
], ignore_index=True)

# Sort final dataset by region, season, lead_month, and year
final_data.sort_values(by=["region", "season", "lead_month", "year"], inplace=True)

agg_ENSO_PRISM = final_data.copy()

output_file = "agg_ENSO_PRISM.csv"
agg_ENSO_PRISM.to_csv(output_file, index=False)


**Load ERA5 and seasonal ECMWF climate variables: TPRATE, TCLW, TCWV, Q850**

In [16]:
clim_vars = ["q", "tclw", "tcwv", "tprate"]

# forecast ECMWF variables
forecast_data = pd.read_csv("5_combined_forecast_variables.csv")
forecast_data = forecast_data.rename(columns={"init_Month": "lead_month", "Region": "region"})
forecast_data["lead_month"] = pd.to_numeric(forecast_data["lead_month"], errors='coerce')

#########################################################################################################
# ERA reanalysis climate variables

df_prism_wt["lead_month"] = pd.to_numeric(df_prism_wt["lead_month"], errors='coerce')

PRISM_CLIMAVAR = df_prism_wt.copy()

# Process each predictor dynamically
for var in clim_vars:
    file_path = f"ERA_{var}_regional.csv"

    if os.path.exists(file_path):
        era_data = pd.read_csv(file_path)
        era_data = era_data.rename(columns={"Region": "region"})

        # Rename "tp" to "tprate" if it exists
        if "tp" in era_data.columns:
            era_data["tp"] = era_data["tp"] / 86400  # Convert from total precipitation (m) to rate (m/s)
            era_data.rename(columns={"tp": "tprate"}, inplace=True)

        # Merge ERA data with weather data for observations
        PRISM_CLIMAVAR = PRISM_CLIMAVAR.merge(
            era_data[["Year", "Month", "region", var]],
            on=["Year", "Month", "region"],
            how="left"
        )

        # Rename observation columns dynamically
        var_obs = f"{var}_obs"
        PRISM_CLIMAVAR.rename(columns={var: var_obs}, inplace=True)

        # Merge with forecast data
        PRISM_CLIMAVAR = PRISM_CLIMAVAR.merge(
            forecast_data[["Year", "Month", "region", "lead_month", var]],
            on=["Year", "Month", "region", "lead_month"],
            how="left"
        )

        # Rename forecast columns dynamically
        var_for = f"{var}_for"
        PRISM_CLIMAVAR.rename(columns={var: var_for}, inplace=True)
    else:
        print(f"Warning: File not found: {file_path}")

print("✅ PRISM_CLIMAVAR successfully created with all variables.")

✅ PRISM_CLIMAVAR successfully created with all variables.


**Seasonal climate features**

In [17]:
# Copy and clean column names
df = PRISM_CLIMAVAR.copy()
df.columns = df.columns.str.lower()

# Define season filtering conditions
'''
season_definitions = {
    "Jun": (df["lead_month"].isin([4, 5, 6])) & (df["month"] == 6),
    "Jul": (df["lead_month"].isin([4, 5, 6, 7])) & (df["month"] == 7),
    "Aug": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"] == 8),
    "Sep": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"] == 9),
    "Oct": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"] == 10),
    "JJASO": (df["lead_month"].isin([4, 5, 6])) & (df["month"].isin([6, 7, 8, 9, 10])),
    "JJA": (df["lead_month"].isin([4, 5, 6])) & (df["month"].isin([6, 7, 8])),
    "JJ": (df["lead_month"].isin([4, 5, 6])) & (df["month"].isin([6, 7])),
    "JASO": (df["lead_month"] == 7) & (df["month"].isin([7, 8, 9, 10])),
    "JAS": (df["lead_month"].isin([4, 5, 6, 7])) & (df["month"].isin([7, 8, 9])),
    "JA": (df["lead_month"].isin([4, 5, 6, 7])) & (df["month"].isin([7, 8])),
    "ASO": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"].isin([8, 9, 10])),
    "AS": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"].isin([8, 9])),
    "SO": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"].isin([9, 10])),
}
'''
# Define season filtering conditions (for October forecast lead month 4 was eliminated)
season_definitions = {
    "Jun": (df["lead_month"].isin([4, 5, 6])) & (df["month"] == 6),
    "Jul": (df["lead_month"].isin([4, 5, 6, 7])) & (df["month"] == 7),
    "Aug": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"] == 8),
    "Sep": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"] == 9),
    "Oct": (df["lead_month"].isin([5, 6, 7, 8])) & (df["month"] == 10),
    "JJASO": (df["lead_month"].isin([5, 6])) & (df["month"].isin([6, 7, 8, 9, 10])),
    "JJA": (df["lead_month"].isin([4, 5, 6])) & (df["month"].isin([6, 7, 8])),
    "JJ": (df["lead_month"].isin([4, 5, 6])) & (df["month"].isin([6, 7])),
    "JASO": (df["lead_month"].isin([4, 5, 6, 7])) & (df["month"].isin([7, 8, 9, 10])),
    "JAS": (df["lead_month"].isin([4, 5, 6, 7])) & (df["month"].isin([7, 8, 9])),
    "JA": (df["lead_month"].isin([4, 5, 6, 7])) & (df["month"].isin([7, 8])),
    "ASO": (df["lead_month"].isin([5, 6, 7, 8])) & (df["month"].isin([8, 9, 10])),
    "AS": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"].isin([8, 9])),
    "SO": (df["lead_month"].isin([5, 6, 7, 8])) & (df["month"].isin([9, 10])),
}

# Create final container for all aggregated results
final_data = pd.DataFrame()

# Loop over all seasons
for season, condition in season_definitions.items():
    season_data = df[condition].copy()
    season_data["season"] = season

    # Dynamically build aggregation dictionary
    agg_dict = {}
    for var in clim_vars:
        for suffix in ["_for", "_obs"]:
            col = f"{var}{suffix}"
            if col in season_data.columns:
                agg_dict[col] = "sum"
    if "areal_mean_precipitation_mm_day" in season_data.columns:
        agg_dict["areal_mean_precipitation_mm_day"] = "mean"

    # Group and aggregate
    aggregated = season_data.groupby(["year", "region", "season", "lead_month"], as_index=False).agg(agg_dict)

    # Rename columns after aggregation
    rename_dict = {
        col: f"sum{col}" for col in agg_dict if col.endswith("_for") or col.endswith("_obs")
    }
    if "areal_mean_precipitation_mm_day" in agg_dict:
        rename_dict["areal_mean_precipitation_mm_day"] = "avgPCP_mmDay"

    aggregated.rename(columns=rename_dict, inplace=True)

    # Append to final data
    final_data = pd.concat([final_data, aggregated], ignore_index=True)

# Standardize _for and _obs variables (Z-score by region & season)
for col in [c for c in final_data.columns if c.endswith("_for") or c.endswith("_obs")]:
    final_data[f"sd_{col}"] = final_data.groupby(["region", "season"])[col].transform(
        lambda x: (x - x.mean()) / x.std() if x.std() != 0 else 0
    )

# Final sort
agg_PRISM_CLIMAVAR = final_data.sort_values(by=["region", "season", "lead_month", "year"]).copy()

# Save to CSV
agg_PRISM_CLIMAVAR.to_csv(f"agg_PRISM_CLIMAVAR_{'_'.join(clim_vars)}.csv", index=False)

## **Observed Antecedent Precipitation (era5)**

In [19]:
# Load the ERA5 reanalysis data
era5_file_path = "ERA_tprate_regional.csv"
era5_tprate = pd.read_csv(era5_file_path)
era5_tprate["tp"] = era5_tprate["tp"] / 86400  # Convert from total precipitation (m) to rate (m/s)
# Load the forecasted precipitation data
forecast_var_path = "5_combined_forecast_variables.csv"  # Ensure correct file path
forecast_data = pd.read_csv(forecast_var_path)

# Convert 'time' column in ERA5 data to datetime format for proper indexing
era5_tprate['time'] = pd.to_datetime(era5_tprate['time'])

# Define initialization months and corresponding target months
init_months = [4, 5, 6, 7, 8]  # April to August
target_months_by_init = {
    4: [6, 7, 8, 9, 10],  # April init → June to October
    5: [6,7, 8, 9, 10],     # May init → July to October
    6: [6,7, 8, 9, 10],        # June init → August to October
    7: [7, 8, 9, 10],           # July init → September to October
    8: [8, 9, 10]               # August init → October
}

# Initialize list to store historical antecedent precipitation
historical_antecedent_data = []

# Get unique years and regions from ERA5 data
years = historic_years
regions = era5_tprate["Region"].unique()

# Loop through each year and initialization month
for year in years:
    for init_month in init_months:
        # Get ERA5 data for this year
        era5_subset = era5_tprate[era5_tprate["Year"] == year]

        for target_month in target_months_by_init[init_month]:  # Only valid target months
            for region in regions:
                # Filter region-specific data
                region_data = era5_subset[era5_subset["Region"] == region]

                # Select antecedent months
                if init_month == 4:  # April Init → Jan, Feb, Mar
                    months = [1, 2, 3]
                elif init_month == 5:  # May Init → Feb, Mar, Apr
                    months = [2, 3, 4]
                elif init_month == 6:  # June Init → Mar, Apr, May
                    months = [3, 4, 5]
                elif init_month == 7:  # July Init → Apr, May, June
                    months = [4, 5, 6]
                elif init_month == 8:  # August Init → May, June, July
                    months = [5, 6, 7]
                else:
                    continue  # Skip invalid cases

                # Extract precipitation values for the selected months
                precip_values = region_data[region_data["Month"].isin(months)]["tp"].values

                # Ensure we have enough data
                if len(precip_values) < 3:
                    antecedent_3m, antecedent_2m, antecedent_1m = np.nan, np.nan, np.nan
                else:
                    antecedent_3m = np.mean(precip_values)  # Average over 3 months
                    antecedent_2m = np.mean(precip_values[1:])  # Average over last 2 months
                    antecedent_1m = precip_values[-1]  # Last month's value

                # Store results
                historical_antecedent_data.append({
                    "Year": year,
                    "Init_Month": init_month,
                    "Target_Month": target_month,
                    "Region": region,
                    "Antecedent_3M": antecedent_3m,
                    "Antecedent_2M": antecedent_2m,
                    "Antecedent_1M": antecedent_1m
                })

# Convert to DataFrame
historic_antecedent_precip = pd.DataFrame(historical_antecedent_data)

# Save the dataset
historic_antecedent_precip.to_csv("historic_antecedent_precip.csv", index=False)

## **Hindcast Antecedent Precipitation**

In [20]:
# Convert 'time' column in ERA5 data to datetime format for proper indexing
era5_tprate['time'] = pd.to_datetime(era5_tprate['time'])
era5_tprate["tp"] = era5_tprate["tp"]   # Convert from total precipitation (m) to rate (m/s)
# Define initialization months and corresponding target months
init_months = [4, 5, 6, 7, 8]  # April to August
target_months_by_init = {
    4: [6, 7, 8, 9,10],  # April init → June to October
    5: [6,7, 8, 9, 10],  # May init → June to October
    6: [6,7, 8, 9, 10],  # June init →cf June to October
    7: [7, 8, 9, 10],  # July init → July to October
    8: [8, 9, 10]  # August init → August to October
}

# Get unique years and regions from ERA5 data
hindcast_years = forecast_data["Year"].unique()
regions = era5_tprate["Region"].unique()

# Initialize a list to store antecedent precipitation data
antecedent_precip_data = []

# Loop through each year in the hindcast dataset
for year in hindcast_years:
    for init_month in init_months:
        if init_month not in target_months_by_init:
            continue  # Skip if no valid target months for this init month

        for target_month in target_months_by_init[init_month]:  # Only valid target months
            for region in regions:
                # Get forecasted data for this region, initialized in the given month
                forecast_subset = forecast_data[
                    (forecast_data["init_Month"] == init_month) &
                    (forecast_data["Year"] == year) &
                    (forecast_data["Region"] == region)
                ]

                # Get ERA5 data for this region for the given year
                era5_subset = era5_tprate[
                    (era5_tprate["Year"] == year) &
                    (era5_tprate["Region"] == region)
                ]

                # Identify antecedent months
                antecedent_3m_months = [target_month - 3, target_month - 2, target_month - 1]
                antecedent_2m_months = [target_month - 2, target_month - 1]
                antecedent_1m_month = target_month - 1  # Usually from forecast

                # Get precipitation values for each case
                precip_values_3m = []
                precip_values_2m = []
                precip_value_1m = np.nan

                for month in antecedent_3m_months:
                    if month < init_month:  # Use ERA5 for months before initialization
                        value = era5_subset.loc[era5_subset["Month"] == month, "tp"]
                    else:  # Use Forecast for months after initialization
                        value = forecast_subset.loc[forecast_subset["Month"] == month, "tprate"]

                    if not value.empty:
                        precip_values_3m.append(value.values[0])

                for month in antecedent_2m_months:
                    if month < init_month:  # Use ERA5 for months before initialization
                        value = era5_subset.loc[era5_subset["Month"] == month, "tp"]
                    else:  # Use Forecast for months after initialization
                        value = forecast_subset.loc[forecast_subset["Month"] == month, "tprate"]

                    if not value.empty:
                        precip_values_2m.append(value.values[0])

                # Assign 1-month antecedent value (default from forecast)
                value_1m = forecast_subset.loc[forecast_subset["Month"] == antecedent_1m_month, "tprate"]
                if not value_1m.empty:
                    precip_value_1m = value_1m.values[0]

                # **NEW FIX**: If `Target Month == Init Month`, use ERA5 from `Target Month - 1`
                if target_month == init_month:
                    era5_value_1m = era5_subset.loc[era5_subset["Month"] == antecedent_1m_month, "tp"]
                    if not era5_value_1m.empty:
                        precip_value_1m = era5_value_1m.values[0]  # Fill from ERA5

                # Compute means
                antecedent_3m = np.nan if len(precip_values_3m) < 3 else np.mean(precip_values_3m)
                antecedent_2m = np.nan if len(precip_values_2m) < 2 else np.mean(precip_values_2m)
                antecedent_1m = precip_value_1m  # Direct value

                # Store the results
                antecedent_precip_data.append({
                    "Year": year,
                    "Init_Month": init_month,
                    "Target_Month": target_month,
                    "Region": region,
                    "Antecedent_3M": antecedent_3m,
                    "Antecedent_2M": antecedent_2m,
                    "Antecedent_1M": antecedent_1m
                })

# Convert to DataFrame
hindcast_antecedent_precip = pd.DataFrame(antecedent_precip_data)

# Save the dataset
hindcast_antecedent_precip.to_csv("hindcast_antecedent_precip.csv", index=False)

print("✅ Missing Antecedent_1M values filled with ERA5 for same Target and Init Month cases!")

✅ Missing Antecedent_1M values filled with ERA5 for same Target and Init Month cases!


## **Combine observed and hindcast antecedent precipitation in one data file.**

In [21]:
# Merge the hindcast and historic antecedent data on Year, Init_Month, Target_Month, and Region
# Perform a full outer merge to keep all data
combined_antecedent_precip = hindcast_antecedent_precip.merge(
    historic_antecedent_precip,
    on=["Year", "Init_Month", "Target_Month", "Region"],
    how="outer",  # Keep all data from both datasets
    suffixes=("_for", "_obs")  # Ensuring correct naming of columns
)

file_path = "combined_antecedent_precip.csv"
combined_antecedent_precip.to_csv(file_path, index=False)

print(f"✅ Combined antecedent precipitation dataset saved as: {file_path}")

✅ Combined antecedent precipitation dataset saved as: combined_antecedent_precip.csv


In [22]:

combined_antecedent_precip.rename(columns={'Target_Month': 'Month'}, inplace=True)
combined_antecedent_precip.rename(columns={'Init_Month': 'lead_month'}, inplace=True)

# Merge df_prism_wt with combined_antecedent_precip on 'Year', 'Month', and 'region'
antecedent_precip_prism = df_prism_wt.merge(
    combined_antecedent_precip,
    left_on=['lead_month', 'Year', 'Month', 'region'],
    right_on=['lead_month','Year', 'Month', 'Region'],
    how='left',
    suffixes=('', '_del')
).drop(columns=['Region'])


# Remove columns that have the suffix '_del'
columns_to_keep = [col for col in antecedent_precip_prism.columns if not col.endswith('_del')]
antecedent_precip_prism = antecedent_precip_prism[columns_to_keep]

In [23]:
df = antecedent_precip_prism.copy()
df.columns = df.columns.str.lower()

# Define seasons
season_definitions = {
    "Jun": df["lead_month"].isin([4, 5, 6]) & (df["month"] == 6),
    "Jul": df["lead_month"].isin([4, 5, 6, 7]) & (df["month"] == 7),
    "Aug": df["lead_month"].isin([4, 5, 6, 7, 8]) & (df["month"] == 8),
    "Sep": df["lead_month"].isin([4, 5, 6, 7, 8]) & (df["month"] == 9),
    "Oct": df["lead_month"].isin([4, 5, 6, 7, 8]) & (df["month"] == 10)
}
final_data = pd.DataFrame()
# Rename mapping
rename_dict = {
    "areal_mean_precipitation_mm_day": "avgPCP_mmDay"
}

# Process seasons, rename, and concatenate
final_data = pd.concat([
    df[condition].assign(season=season).rename(columns={k: v for k, v in rename_dict.items() if k in df.columns})
    for season, condition in season_definitions.items()
], ignore_index=True)


# Sort dataset
antecedent_precip_prism_df = final_data.sort_values(by=["region", "season", "lead_month", "year"]).copy()

In [24]:
df = df_prism_wt.copy()

# Convert column names to lowercase
df.columns = df.columns.str.lower()

# Define seasons
season_definitions = {
    "Jun": (df["lead_month"].isin([4, 5, 6])) & (df["month"] == 6),
    "Jul": (df["lead_month"].isin([4, 5, 6, 7])) & (df["month"] == 7),
    "Aug": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"] == 8),
    "Sep": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"] == 9),
    "Oct": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"] == 10),
    "JJASO": (df["lead_month"].isin([4, 5, 6])) & (df["month"].isin([6, 7, 8, 9, 10])),
    "JJA": (df["lead_month"].isin([4, 5, 6])) & (df["month"].isin([6, 7, 8])),
    "JJ": (df["lead_month"].isin([4, 5, 6])) & (df["month"].isin([6, 7])),
    "JASO": (df["lead_month"].isin([4, 5, 6, 7])) & (df["month"].isin([7, 8, 9, 10])),
    "JAS": (df["lead_month"].isin([4, 5, 6, 7])) & (df["month"].isin([7, 8, 9])),
    "JA": (df["lead_month"].isin([4, 5, 6, 7])) & (df["month"].isin([7, 8])),
    "ASO": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"].isin([8, 9, 10])),
    "AS": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"].isin([8, 9])),
    "SO": (df["lead_month"].isin([4, 5, 6, 7, 8])) & (df["month"].isin([9, 10])),
}

final_data = pd.DataFrame()

for season, condition in season_definitions.items():
    # Filter dataset for current season
    season_data = df[condition].copy()

    season_data["season"] = season

    # Aggregate data using groupby
    aggregated = season_data.groupby(["year", "region", "season", "lead_month"], as_index=False).agg({
        "dry.for": "sum",
        "normal.for": "sum",
        "monsoon.for": "sum",
        "dry.obs": "sum",
        "normal.obs": "sum",
        "monsoon.obs": "sum",
        "areal_mean_precipitation_mm_day": "mean"
    })

    # Fix multi-index column names (Flatten multi-index)
    aggregated.columns = ["_".join(col).strip("_") if isinstance(col, tuple) else col for col in aggregated.columns]

    # Rename columns to match the required output format
    aggregated.rename(columns={
        "dry.for": "sumDry.for",
        "monsoon.for": "sumMonsoon.for",
        "dry.obs": "sumDry.obs",
        "monsoon.obs": "sumMonsoon.obs",
        "areal_mean_precipitation_mm_day": "avgPCP_mmDay",  # Mean precipitation
}, inplace=True)

    final_data = pd.concat([final_data, aggregated], ignore_index=True)

# Sort final dataset by region, season, lead_month, and year
final_data.sort_values(by=["region", "season", "lead_month", "year"], inplace=True)

agg_WT_PRISM = final_data

In [25]:
antecedent_precip_prism_WT = pd.DataFrame()
antecedent_precip_prism_WT = agg_WT_PRISM.merge(
    antecedent_precip_prism_df,
    on=['lead_month', 'year', 'region', 'season'],
    how='left',
    suffixes=('', '_del')  # Adding suffix for duplicate column identification
)

# Remove columns that have the suffix '_del'
columns_to_keep = [col for col in antecedent_precip_prism_WT.columns if not col.endswith('_del')]

# Keep only the necessary columns
antecedent_precip_prism_WT = antecedent_precip_prism_WT[columns_to_keep]
antecedent_precip_prism_df.to_csv("antecedent_precip_prism_df.csv", index=False)

In [26]:
agg_WT_PRISM.to_csv("paired_allSeasons_ECMWF_PRISM_WT_1981_2023.csv", index=False)

# Combine Predictors

In [27]:
import pandas as pd
import os

# File paths
base_path = ""
file_names = [
    "paired_allSeasons_ECMWF_PRISM_WT_1981_2023.csv",
    "antecedent_precip_prism_df.csv",
    "agg_PRISM_CLIMAVAR_q_tclw_tcwv_tprate.csv",
    "agg_ENSO_PRISM.csv"
]

# Load files only if they exist
dataframes = {}
for file_name in file_names:
    full_path = os.path.join(base_path, file_name)
    if os.path.exists(full_path):
        df_name = os.path.splitext(file_name)[0].replace(" ", "_").replace("(", "").replace(")", "")
        dataframes[df_name] = pd.read_csv(full_path)

# Start with the base dataset
main_key = "paired_allSeasons_ECMWF_PRISM_WT_1981_2023"
merged_df = dataframes[main_key]

# Columns to exclude if they appear in any file
excluded_cols = {"yyyymm", "dry.obs", "normal.obs", "monsoon.obs", "month", "lead_date",
                 "dry.for", "normal.for", "monsoon.for", "date", "avgPCP_mmDay"}

# Merge other datasets on ['region', 'season', 'year', 'lead_month']
merge_keys = ["region", "season", "year", "lead_month"]
for key, df in dataframes.items():
    if key != main_key:
        cols_to_use = [col for col in df.columns if col not in excluded_cols]
        df_filtered = df[merge_keys + [col for col in cols_to_use if col not in merge_keys]]
        merged_df = pd.merge(merged_df, df_filtered, on=merge_keys, how="left")

# Drop columns containing "sd"
merged_df = merged_df.loc[:, ~merged_df.columns.str.contains("sd")]
# Rename _for and _obs to .for and .obs
merged_df.columns = merged_df.columns.str.replace("_for", ".for").str.replace("_obs", ".obs")

# Save the final cleaned merged dataset
output_path = "Predictors.csv"
merged_df.to_csv(output_path, index=False)
output_path



'Predictors.csv'

Fill seasonal antecedent precipitation and ENSO predictors using the monthly values.

In [30]:
import pandas as pd

# Load the dataset
Predictors = pd.read_csv("Predictors.csv")

# Update column names to use _for and _obs instead of .for and .obs
Predictors.columns = [col.replace(".for", "_for").replace(".obs", "_obs") for col in Predictors.columns]

# Update the list of antecedent and ENSO-related columns
antecedent_cols_extended = [
    "antecedent_3m_for", "antecedent_2m_for", "antecedent_1m_for",
    "antecedent_3m_obs", "antecedent_2m_obs", "antecedent_1m_obs",
    "ant_dsst3_4_for", "ant_mei_for", "ant_soi_for",
    "ant_dsst3_4_obs", "ant_mei_obs", "ant_soi_obs"
]

# Define season aggregation mapping
agg_season_mapping = {
    "JJ": "Jun", "JA": "Jun", "AS": "Aug", "SO": "Sep",
    "JJA": "Jun", "JAS": "Jul", "ASO": "Aug", "JASO": "Jul", "JJASO": "Jun"
}

# Apply the fill logic
for agg_season, base_month in agg_season_mapping.items():
    for (region, year), group in Predictors.groupby(["region", "year"]):
        agg_mask = (Predictors["region"] == region) & \
                   (Predictors["year"] == year) & \
                   (Predictors["season"] == agg_season)
        base_mask = (Predictors["region"] == region) & \
                    (Predictors["year"] == year) & \
                    (Predictors["season"] == base_month)

        if agg_mask.any() and base_mask.any():
            for col in antecedent_cols_extended:
                if col in Predictors.columns:
                    if Predictors.loc[agg_mask, col].isna().all() and Predictors.loc[base_mask, col].notna().any():
                        Predictors.loc[agg_mask, col] = Predictors.loc[base_mask, col].values[0]
Predictors_filled = Predictors
# Save the cleaned dataset
Predictors_filled.to_csv("Predictors_filled.csv", index=False)
